<a href="https://colab.research.google.com/github/kinjaljoshi/s2t-training/blob/main/big_query_testing_iter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [ ]:
!pip install google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = 'llm-text-to-sql-445914'
!gcloud config set project $PROJECT_ID

Updated property [core/project].


# Install and import

In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import PeftModel
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer
import torch, re
import pandas as pd
import nltk


In [ ]:
!!pip uninstall -y shapely
!pip install shapely --upgrade
!pip install google-cloud-bigquery --upgrade
import nltk
nltk.download('punkt', force=True)
nltk.download('punkt_tab', force=True)
print(nltk.find('tokenizers/punkt'))
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import PeftModel
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer
import torch, re
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.7/243.7 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


/root/nltk_data/tokenizers/punkt


# **Create DB Table and Insert Date**

In [ ]:
from google.cloud import bigquery

# Set up BigQuery client
client = bigquery.Client()

# Define the dataset and table name
dataset_id = "llm-text-to-sql-445914.llm_text_to_sql"  # Replace with your dataset ID
table_id = "RX_SALES_TRANSACTIONS"

table_ref = f"{dataset_id}.{table_id}"

# Define the schema for the table
schema = [
    bigquery.SchemaField("TRX_ID", "INTEGER", description="Unique Sales transaction identifier"),
    bigquery.SchemaField("SAL_WHS", "INTEGER", description="Warehouse Identifier"),
    bigquery.SchemaField("SAL_WHN", "STRING", description="Warehouse Name"),
    bigquery.SchemaField("SAL_WHL", "STRING", description="Warehouse Region"),
    bigquery.SchemaField("SAL_WNC", "STRING", description="Warehouse City"),
    bigquery.SchemaField("SAL_WHA_ONE", "STRING", description="Warehouse Building Address"),
    bigquery.SchemaField("SAL_WHA_TWO", "STRING", description="Warehouse Street Address"),
    bigquery.SchemaField("SAL_WHA_TRE", "STRING", description="Warehouse Address City"),
    bigquery.SchemaField("SAL_WHA_FOR", "STRING", description="Warehouse Address State"),
    bigquery.SchemaField("SAL_WHA_FIV", "STRING", description="Warehouse Address Country"),
    bigquery.SchemaField("SAL_WHA_ZIP", "STRING", description="Warehouse Address zip code"),
    bigquery.SchemaField("SAL_WHA_MGR", "STRING", description="Warehouse Manager Identifier"),
    bigquery.SchemaField("SAL_WHA_MGAF", "STRING", description="Warehouse Manager First Name"),
    bigquery.SchemaField("SAL_WHA_MGAL", "STRING", description="Warehouse Manager Last Name"),
    bigquery.SchemaField("SAL_SALES_EID", "INTEGER", description="Sales person Employee Identifier"),
    bigquery.SchemaField("SAL_SALES_EFN", "STRING", description="Sales person First Name"),
    bigquery.SchemaField("SAL_SALES_ELN", "STRING", description="Sales person Last Name"),
    bigquery.SchemaField("SAL_SALES_MID", "STRING", description="Sales person Manager Identifier"),
    bigquery.SchemaField("SAL_SALES_MIF", "STRING", description="Sales person Manager First Name"),
    bigquery.SchemaField("SAL_SALES_MIL", "STRING", description="Sales person Manager Last Name"),
    bigquery.SchemaField("SAL_SALES_CP", "STRING", description="Sales person commission percent"),
    bigquery.SchemaField("SAL_SALES_HD", "DATE", description="Sales person hire date"),
    bigquery.SchemaField("SAL_SSIC", "INTEGER", description="Sale item unique identifier"),
    bigquery.SchemaField("SAL_ITMN", "STRING", description="Sale item number used for ordering by customer"),
    bigquery.SchemaField("SAL_ITMNME", "STRING", description="Sale item Name"),
    bigquery.SchemaField("SAL_ITMDE", "STRING", description="Sale item description"),
    bigquery.SchemaField("SAL_ITEM_UOM", "STRING", description="Sale item Unit of measure"),
    bigquery.SchemaField("SAL_ITEM_WT", "FLOAT", description="Sale item weight in kilogram"),
    bigquery.SchemaField("SAL_ITEM_L", "FLOAT", description="Sale item Length measurement in centimeter"),
    bigquery.SchemaField("SAL_ITEM_W", "FLOAT", description="Sale item Width measurement in centimeter"),
    bigquery.SchemaField("SAL_ITEM_H", "FLOAT", description="Sale item Height measurement in centimeter"),
    bigquery.SchemaField("SAL_ITEM_DISP", "STRING", description="Sale Item Disposition ('H', 'N')"),
    bigquery.SchemaField("SAL_ITEM_DISPD", "STRING", description="Sale item disposition detail (Hazard or None)"),
    bigquery.SchemaField("SAL_INV_ID", "INTEGER", description="Sale invoice unique identifier"),
    bigquery.SchemaField("SAL_INV_DT", "DATE", description="Sale invoice creation date"),
    bigquery.SchemaField("SAL_INV_AMT", "FLOAT", description="Sale invoice amount"),
    bigquery.SchemaField("SAL_INV_PDT", "DATE", description="Sale invoice expected payment date"),
    bigquery.SchemaField("SAL_INVOICE_STS", "STRING", description="Sale invoice status ('O', 'C')"),
    bigquery.SchemaField("SAL_INV_DISCA", "FLOAT", description="Sale invoice discount amount"),
    bigquery.SchemaField("SAL_INV_ADJA", "STRING", description="Sale invoice adjusted amount"),
    bigquery.SchemaField("SAL_INV_CLD", "STRING", description="Sale invoice closed date"),
    bigquery.SchemaField("SAL_CUST_ID", "INTEGER", description="Sale Customer Identifier"),
    bigquery.SchemaField("SAL_CUST_ACCT", "STRING", description="Sale Customer Account Identifier"),
    bigquery.SchemaField("SAL_CUST_NUM", "STRING", description="Sale Customer account number"),
    bigquery.SchemaField("SAL_CUST_A1", "STRING", description="Sale Customer Billing address House number"),
    bigquery.SchemaField("SAL_CUST_A2", "STRING", description="Sale Customer Billing address Street"),
    bigquery.SchemaField("SAL_CUST_A3", "STRING", description="Sale Customer Billing address city"),
    bigquery.SchemaField("SAL_CUST_A4", "STRING", description="Sale Customer Billing address state"),
    bigquery.SchemaField("SAL_CUST_A5", "STRING", description="Sale Customer Billing address Country"),
    bigquery.SchemaField("SAL_CUST_A6", "STRING", description="Sale Customer Billing address zip code"),
    bigquery.SchemaField("SAL_CUST_B1", "STRING", description="Sale Customer Shipping address House number"),
    bigquery.SchemaField("SAL_CUST_B2", "STRING", description="Sale Customer Shipping address Street"),
    bigquery.SchemaField("SAL_CUST_B3", "STRING", description="Sale Customer Shipping address city"),
    bigquery.SchemaField("SAL_CUST_B4", "STRING", description="Sale Customer Shipping address state"),
    bigquery.SchemaField("SAL_CUST_B5", "STRING", description="Sale Customer Shipping address Country"),
    bigquery.SchemaField("SAL_CUST_B6", "STRING", description="Sale Customer Shipping address zip code"),
    bigquery.SchemaField("SAL_QTY", "INTEGER", description="Sale quantity"),
    bigquery.SchemaField("SAL_DATE", "DATE", description="Date of Sale"),
    bigquery.SchemaField("SAL_DEL_DATE", "DATE", description="Delivery date of sale"),
]

# Create the table
try:
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)  # API request
    print(f"Table {table.table_id} created successfully in dataset {dataset_id}.")
except Exception as e:
    print(f"An error occurred: {e}")

Table RX_SALES_TRANSACTIONS created successfully in dataset llm-text-to-sql-445914.llm_text_to_sql.


In [ ]:
from google.cloud import bigquery
import random
from datetime import datetime, timedelta

# Set up BigQuery client
client = bigquery.Client()

# Define the dataset and table name
dataset_id = "llm-text-to-sql-445914.llm_text_to_sql"  # Replace with your dataset ID
table_id = "RX_SALES_TRANSACTIONS"

table_ref = f"{dataset_id}.{table_id}"

# Generate sample data for 20 records
def generate_sample_data():
    records = []
    for i in range(1, 21):
        record = {
            "TRX_ID": i,
            "SAL_WHS": random.randint(1, 10),
            "SAL_WHN": f"Warehouse_{random.randint(1, 10)}",
            "SAL_WHL": random.choice(["North", "South", "East", "West"]),
            "SAL_WNC": random.choice(["City_A", "City_B", "City_C"]),
            "SAL_WHA_ONE": f"Building_{random.randint(1, 100)}",
            "SAL_WHA_TWO": f"Street_{random.randint(1, 50)}",
            "SAL_WHA_TRE": random.choice(["City_A", "City_B", "City_C"]),
            "SAL_WHA_FOR": random.choice(["State_X", "State_Y", "State_Z"]),
            "SAL_WHA_FIV": "Country_ABC",
            "SAL_WHA_ZIP": f"{random.randint(10000, 99999)}",
            "SAL_WHA_MGR": f"MGR_{random.randint(1, 50)}",
            "SAL_WHA_MGAF": f"Manager_First_{random.randint(1, 50)}",
            "SAL_WHA_MGAL": f"Manager_Last_{random.randint(1, 50)}",
            "SAL_SALES_EID": random.randint(1, 100),
            "SAL_SALES_EFN": f"Employee_First_{random.randint(1, 100)}",
            "SAL_SALES_ELN": f"Employee_Last_{random.randint(1, 100)}",
            "SAL_SALES_MID": f"MID_{random.randint(1, 50)}",
            "SAL_SALES_MIF": f"Manager_First_{random.randint(1, 50)}",
            "SAL_SALES_MIL": f"Manager_Last_{random.randint(1, 50)}",
            "SAL_SALES_CP": f"{random.randint(1, 20)}%",
            "SAL_SALES_HD": (datetime.now() - timedelta(days=random.randint(0, 3650))).date().isoformat(),
            "SAL_SSIC": random.randint(1, 1000),
            "SAL_ITMN": f"Item_{random.randint(1, 100)}",
            "SAL_ITMNME": f"Item_Name_{random.randint(1, 100)}",
            "SAL_ITMDE": f"Item_Description_{random.randint(1, 100)}",
            "SAL_ITEM_UOM": random.choice(["KG", "L", "CM"]),
            "SAL_ITEM_WT": random.uniform(1.0, 100.0),
            "SAL_ITEM_L": random.uniform(10.0, 200.0),
            "SAL_ITEM_W": random.uniform(10.0, 200.0),
            "SAL_ITEM_H": random.uniform(10.0, 200.0),
            "SAL_ITEM_DISP": random.choice(["H", "N"]),
            "SAL_ITEM_DISPD": random.choice(["Hazard", "None"]),
            "SAL_INV_ID": random.randint(1, 500),
            "SAL_INV_DT": (datetime.now() - timedelta(days=random.randint(0, 1000))).date().isoformat(),
            "SAL_INV_AMT": random.uniform(100.0, 10000.0),
            "SAL_INV_PDT": (datetime.now() + timedelta(days=random.randint(0, 100))).date().isoformat(),
            "SAL_INVOICE_STS": random.choice(["O", "C"]),
            "SAL_INV_DISCA": random.uniform(0.0, 500.0),
            "SAL_INV_ADJA": f"{random.uniform(0.0, 500.0):.2f}",
            "SAL_INV_CLD": (datetime.now() - timedelta(days=random.randint(0, 1000))).date().isoformat(),
            "SAL_CUST_ID": random.randint(1, 1000),
            "SAL_CUST_ACCT": f"Account_{random.randint(1, 100)}",
            "SAL_CUST_NUM": f"CustNum_{random.randint(1, 100)}",
            "SAL_CUST_A1": f"House_{random.randint(1, 100)}",
            "SAL_CUST_A2": f"Street_{random.randint(1, 50)}",
            "SAL_CUST_A3": random.choice(["City_A", "City_B", "City_C"]),
            "SAL_CUST_A4": random.choice(["State_X", "State_Y", "State_Z"]),
            "SAL_CUST_A5": "Country_ABC",
            "SAL_CUST_A6": f"{random.randint(10000, 99999)}",
            "SAL_CUST_B1": f"House_{random.randint(1, 100)}",
            "SAL_CUST_B2": f"Street_{random.randint(1, 50)}",
            "SAL_CUST_B3": random.choice(["City_A", "City_B", "City_C"]),
            "SAL_CUST_B4": random.choice(["State_X", "State_Y", "State_Z"]),
            "SAL_CUST_B5": "Country_ABC",
            "SAL_CUST_B6": f"{random.randint(10000, 99999)}",
            "SAL_QTY": random.randint(1, 100),
            "SAL_DATE": (datetime.now() - timedelta(days=random.randint(0, 1000))).date().isoformat(),
            "SAL_DEL_DATE": (datetime.now() + timedelta(days=random.randint(1, 10))).date().isoformat(),
        }
        records.append(record)
    return records

# Insert the records into the table
records = generate_sample_data()
errors = client.insert_rows_json(table_ref, records)  # API request

if errors:
    print(f"Errors occurred while inserting rows: {errors}")
else:
    print("20 records added successfully to the table.")

20 records added successfully to the table.


# Load and merge model and tokenizer

In [ ]:
!gsutil -m cp -r gs://llm-pipeline-resources/models/iteration-3/iter2_with_react_11_epoch /content/

Copying gs://llm-pipeline-resources/models/iteration-3/iter2_with_react_11_epoch/README.md...
Copying gs://llm-pipeline-resources/models/iteration-3/iter2_with_react_11_epoch/adapter_config.json...
Copying gs://llm-pipeline-resources/models/iteration-3/iter2_with_react_11_epoch/adapter_model.safetensors...
- [3/3 files][ 36.0 MiB/ 36.0 MiB] 100% Done                                    
Operation completed over 3 objects/36.0 MiB.                                     


In [ ]:
base_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
new_model = PeftModel.from_pretrained(base_model, "/content/iter2_with_react_11_epoch")

unload_model = new_model.merge_and_unload()
unload_model.eval()
unload_model.to(device='cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

# Pretokenize and combine and sliding window definitions

In [ ]:
def tokenize_sentences_with_t5(text, tokenizer):
    """
    Tokenizes a long text using sentence tokenization and T5 tokenizer.

    Parameters:
        text (str): The input text to tokenize.
        tokenizer: The T5 tokenizer instance.

    Returns:
        List[int]: A single tokenized string (list of token IDs) prepared for combination.
    """
    # Sentence tokenization
    from nltk.tokenize import sent_tokenize
    sentences = sent_tokenize(text)

    # Tokenize each sentence and combine them
    tokenized_string = []
    for sentence in sentences:
        tokenized_sentence = tokenizer(sentence, add_special_tokens=False)["input_ids"]
        tokenized_string.extend(tokenized_sentence)

        # Add a space token to preserve sentence spacing
        space_token = tokenizer(" ", add_special_tokens=False)["input_ids"]
        if space_token:  # Ensure space_token is not empty
            tokenized_string.extend(space_token)

    # Remove trailing space tokens if any
    if tokenized_string:
        space_token = tokenizer(" ", add_special_tokens=False)["input_ids"]
        if space_token and tokenized_string[-len(space_token):] == space_token:
            tokenized_string = tokenized_string[:-len(space_token)]

    return tokenized_string


In [ ]:
def combine_t5_tokenized_strings(tokens1, tokens2, tokenizer, max_length=None, add_special_tokens=True):
    """
    Combines two T5 tokenized strings into one, optionally adding special tokens
    and truncating to a maximum length.

    Parameters:
        tokens1 (List[int]): The first tokenized string (list of token IDs).
        tokens2 (List[int]): The second tokenized string (list of token IDs).
        tokenizer: The T5 tokenizer instance.
        max_length (int, optional): Maximum allowed length for the combined tokens.
        add_special_tokens (bool): Whether to add special tokens like <cls> and <sep>.

    Returns:
        List[int]: Combined tokenized string.
    """
    combined_tokens = []

    # Add <cls> token if needed
    if add_special_tokens and tokenizer.cls_token_id is not None:
        combined_tokens.append(tokenizer.cls_token_id)

    # Add first tokenized string
    combined_tokens.extend(tokens1)
    #print(f"After first token addition - {combined_tokens}")

    # Add <sep> token if needed
    if add_special_tokens and tokenizer.sep_token_id is not None:
        combined_tokens.append(tokenizer.sep_token_id)

    # Add second tokenized string
    combined_tokens.extend(tokens2)
    #print(f"After Second token addition - {combined_tokens}")

    # Add final <sep> token if needed
    if add_special_tokens and tokenizer.sep_token_id is not None:
        combined_tokens.append(tokenizer.sep_token_id)

    # Truncate if max_length is provided
    if max_length is not None and len(combined_tokens) > max_length:
        combined_tokens = combined_tokens[:max_length]

    return combined_tokens


In [ ]:
def sliding_window_tokenizer(tokenized_string, max_length, stride):
    """
    Splits a large tokenized string into overlapping chunks using a sliding window approach.

    Parameters:
        tokenized_string (List[int]): The tokenized string (list of token IDs).
        max_length (int): Maximum allowed length for each chunk.
        stride (int): Number of tokens to overlap between consecutive chunks.

    Returns:
        List[List[int]]: A list of tokenized chunks, each with a length <= max_length.
    """
    chunks = []
    start = 0

    while start < len(tokenized_string):
        # Define the end of the current window
        end = min(start + max_length, len(tokenized_string))
        # Extract the chunk
        chunk = tokenized_string[start:end]
        chunks.append(chunk)
        # Move the window by the stride
        start += stride

    return chunks


In [ ]:
# def sliding_window_sentence_tokenize(tokenizer, text, max_length=512, stride=256):
#     """
#     Tokenizes text into chunks using a sliding window approach to avoid exceeding the model's max_length.

#     Parameters:
#         tokenizer: The tokenizer instance.
#         text (str): The input text to tokenize.
#         max_length (int): Maximum token length allowed by the model.
#         stride (int): Number of overlapping tokens between chunks.

#     Returns:
#         List[List[int]]: List of tokenized chunks.
#     """
#     # Split text into sentences
#     from nltk.tokenize import sent_tokenize
#     sentences = sent_tokenize(text)

#     # Tokenize sentences and accumulate into chunks
#     chunks = []
#     current_chunk = []
#     current_length = 0

#     for sentence in sentences:
#         tokenized_sentence = tokenizer(sentence, add_special_tokens=False)['input_ids']

#         # Add a space token between sentences to preserve spacing
#         if current_chunk and tokenized_sentence:
#             space_token = tokenizer(" ", add_special_tokens=False)['input_ids']
#             tokenized_sentence = space_token + tokenized_sentence

#         sentence_length = len(tokenized_sentence)

#         # If adding the current sentence exceeds max_length
#         while current_length + sentence_length > max_length:
#             # Fill up the current chunk to max_length
#             available_space = max_length - current_length
#             if available_space > 0:
#                 current_chunk.extend(tokenized_sentence[:available_space])
#                 tokenized_sentence = tokenized_sentence[available_space:]

#             chunks.append(current_chunk)
#             current_chunk = tokenized_sentence[:stride]  # Start new chunk with overlap
#             tokenized_sentence = tokenized_sentence[stride:]
#             current_length = len(current_chunk)

#         # Add remaining tokens to the current chunk
#         current_chunk.extend(tokenized_sentence)
#         current_length += len(tokenized_sentence)

#     # Add the last chunk
#     if current_chunk:
#         chunks.append(current_chunk[:max_length])

#     return chunks

# ReAct Warmup

In [ ]:
react_prompts = [{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all records from the table containing details about employees.",
    "sql_context": "\n#### Table Definitions:\n TABLE  E_MP 'Tracks employee details' (EID 'Unique identifier for an employee', E_FN 'First Name of the employee', E_LN 'Last Name of the employee', E_S 'Indicator if the employee is active or inactive', E_J_D 'Date the employee joined');",
    "reasoning": "The question asks to retrieve all records from the table containing employee details. Since all details are needed, the `SELECT *` statement is appropriate to fetch all columns. The alias `emp` is assigned to the table `E_MP` for readability and ease of reference in complex queries.",
    "action": "SELECT * FROM E_MP emp;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Get full name of all employees.",
    "sql_context": "\n#### Table Definitions:\n TABLE  E_MP 'Tracks employee details' (EID 'Unique identifier for an employee', E_FN 'First Name of the employee', E_LN 'Last Name of the employee', E_S 'Indicator if the employee is active or inactive', E_J_D 'Date the employee joined');",
    "reasoning": "Full name is typically formed by combining first and last names. From the table, the columns `E_FN` and `E_LN` store the first and last names of employees, respectively. These columns are concatenated using the `||` operator to form the full name, and the result is aliased as `employee_full_name` for clarity. The alias `emp` is assigned to the table for readability.",
    "action": "SELECT E_FN || ' ' || E_LN AS employee_full_name FROM E_MP emp;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve the full name and calculate the annual salary of employees from a financial table.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_FNL 'Tracks financial employee details' (EMP_ID 'Unique identifier for an employee', EMP_FN 'First Name of the employee', EMP_LN 'Last Name of the employee', EMP_SAL_MTH 'Monthly salary of the employee', EMP_ACTIVE 'Indicates if the employee is active');",
    "reasoning": "The query requires two components: the full name and the annual salary of employees. The columns `EMP_FN` and `EMP_LN` are selected because they store the first and last names of employees, respectively. These are concatenated using the `||` operator to form the full name. To calculate the annual salary, the column `EMP_SAL_MTH`, which stores the monthly salary, is multiplied by 12. The alias `fin` is assigned to the table for better readability.",
    "action": "SELECT EMP_FN || ' ' || EMP_LN AS employee_full_name, EMP_SAL_MTH * 12 AS annual_salary FROM STT_RCTW_FNL fin;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve the formatted address of transportation hubs.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_TRNS 'Tracks transportation hub details' (HUB_ID 'Unique identifier for a hub', HUB_NAME 'Name of the hub', HUB_STREET 'Street name of the hub', HUB_CITY 'City where the hub is located', HUB_STATE 'State of the hub', HUB_ZIP 'Zip code of the hub');",
    "reasoning": "A formatted address requires components such as street, city, state, and zip code. The columns `HUB_STREET`, `HUB_CITY`, `HUB_STATE`, and `HUB_ZIP` store these respective details about the transportation hub. These columns are concatenated with appropriate separators (e.g., commas and spaces) to form a complete address. The hub name (`HUB_NAME`) is also included for context. The table alias `trns` is used for readability.",
    "action": "SELECT HUB_NAME, HUB_STREET || ', ' || HUB_CITY || ', ' || HUB_STATE || ' ' || HUB_ZIP AS full_address FROM STT_RCTW_TRNS trns;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve transportation hub details and calculate the total number of vehicles serviced annually.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_TRNS 'Tracks transportation hub details' (HUB_ID 'Unique identifier for a hub', HUB_NAME 'Name of the hub', VEHICLES_SERVICED_MTH 'Number of vehicles serviced monthly');",
    "reasoning": "The query requires details of transportation hubs and the annual number of vehicles serviced. The column `HUB_NAME` is included to provide the name of each hub. The column `VEHICLES_SERVICED_MTH`, which tracks the number of vehicles serviced monthly, is multiplied by 12 to calculate the annual total. The alias `trns` is assigned to the table for clarity and consistency.",
    "action": "SELECT HUB_NAME, VEHICLES_SERVICED_MTH * 12 AS vehicles_serviced_annually FROM STT_RCTW_TRNS trns;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve full names of employees and drivers, and their associated vehicle counts.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_TRNS 'Tracks transportation details' (\n     COL_A 'Unique identifier for the record',\n     COL_B 'First name of the employee',\n     COL_C 'Last name of the employee',\n     COL_D 'First name of the driver',\n     COL_E 'Last name of the driver',\n     COL_F 'Number of vehicles assigned to the employee',\n     COL_G 'Number of vehicles driven by the driver'\n );",
    "reasoning": "The query requires the full names of both employees and drivers and their associated vehicle counts. The columns `COL_B` (employee first name) and `COL_C` (employee last name) are concatenated to form the full name of employees. Similarly, `COL_D` (driver first name) and `COL_E` (driver last name) are concatenated to form the full name of drivers. The columns `COL_F` and `COL_G` are selected to represent the number of vehicles assigned to employees and driven by drivers, respectively. The table alias `trns` is used for readability.",
    "action": "SELECT \n    COL_B || ' ' || COL_C AS employee_full_name,\n    COL_D || ' ' || COL_E AS driver_full_name,\n    COL_F AS vehicles_assigned,\n    COL_G AS vehicles_driven\nFROM STT_RCTW_TRNS trns;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve formatted addresses for employees and drivers.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_TRNS 'Tracks transportation records' (\n     COL_A 'Unique identifier for the record',\n     COL_B 'Street address of the employee',\n     COL_C 'City of the employee',\n     COL_D 'State of the employee',\n     COL_E 'Zip code of the employee',\n     COL_F 'Street address of the driver',\n     COL_G 'City of the driver',\n     COL_H 'State of the driver',\n     COL_I 'Zip code of the driver'\n );",
    "reasoning": "To generate formatted addresses, the query requires the street, city, state, and zip code of both employees and drivers. The columns `COL_B`, `COL_C`, `COL_D`, and `COL_E` store these components for employees, and the columns `COL_F`, `COL_G`, `COL_H`, and `COL_I` store them for drivers. These columns are concatenated with appropriate separators (commas and spaces) to create the formatted addresses for both. The table alias `trns` is assigned for clarity.",
    "action": "SELECT \n    COL_B || ', ' || COL_C || ', ' || COL_D || ' ' || COL_E AS employee_address,\n    COL_F || ', ' || COL_G || ', ' || COL_H || ' ' || COL_I AS driver_address\nFROM STT_RCTW_TRNS trns;",
    "final_answer": ""
}
,
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve the formatted billing address of customers.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_CUST 'Tracks customer address details' (\n     COL_A 'Unique identifier for the customer',\n     COL_B 'Street address for billing purposes',\n     COL_C 'City of the billing address',\n     COL_D 'State of the billing address',\n     COL_E 'Zip code of the billing address',\n     COL_F 'Country of the billing address'\n );",
    "reasoning": "The query requires the formatted billing address of customers. The columns `COL_B`, `COL_C`, `COL_D`, `COL_E`, and `COL_F` store the street, city, state, zip code, and country of the billing address, respectively. These components are concatenated with commas and spaces for proper formatting. The table alias `cust` can be assigned for readability in complex queries.",
    "action": "SELECT \n    COL_B || ', ' || COL_C || ', ' || COL_D || ' ' || COL_E || ', ' || COL_F AS billing_address\nFROM STT_RCTW_CUST;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all formatted addresses (billing, shipping, correspondence, dispute) for each customer.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_CUST 'Tracks customer address details' (\n     COL_B 'Street address for billing purposes',\n     COL_C 'City of the billing address',\n     COL_D 'State of the billing address',\n     COL_E 'Zip code of the billing address',\n     COL_F 'Country of the billing address',\n     COL_G 'Street address for shipping purposes',\n     COL_H 'City of the shipping address',\n     COL_I 'State of the shipping address',\n     COL_J 'Zip code of the shipping address',\n     COL_K 'Country of the shipping address',\n     COL_L 'Street address for correspondence purposes',\n     COL_M 'City of the correspondence address',\n     COL_N 'State of the correspondence address',\n     COL_O 'Zip code of the correspondence address',\n     COL_P 'Country of the correspondence address',\n     COL_Q 'Street address for dispute purposes',\n     COL_R 'City of the dispute address',\n     COL_S 'State of the dispute address',\n     COL_T 'Zip code of the dispute address',\n     COL_U 'Country of the dispute address'\n );",
    "reasoning": "The query requires formatted addresses for all purposes: billing, shipping, correspondence, and dispute. Each type of address uses specific columns: `COL_B` to `COL_F` for billing, `COL_G` to `COL_K` for shipping, `COL_L` to `COL_P` for correspondence, and `COL_Q` to `COL_U` for dispute. These columns are concatenated with separators (commas and spaces) to create properly formatted addresses. The table alias `cust` can be assigned for better readability.",
    "action": "SELECT \n    COL_B || ', ' || COL_C || ', ' || COL_D || ' ' || COL_E || ', ' || COL_F AS billing_address,\n    COL_G || ', ' || COL_H || ', ' || COL_I || ' ' || COL_J || ', ' || COL_K AS shipping_address,\n    COL_L || ', ' || COL_M || ', ' || COL_N || ' ' || COL_O || ', ' || COL_P AS correspondence_address,\n    COL_Q || ', ' || COL_R || ', ' || COL_S || ' ' || COL_T || ', ' || COL_U AS dispute_address\nFROM STT_RCTW_CUST;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all records with their corresponding billing date.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Billing date of the transaction'\n );",
    "reasoning": "The query requests all records along with their corresponding billing date. The column `COL_A` is selected because it uniquely identifies each transaction. The column `COL_B` is selected because it stores the billing date for each transaction. Both columns are retrieved as they meet the query requirements. The table alias `bill` can be used for clarity.",
    "action": "SELECT COL_A, COL_B AS billing_date FROM STT_RCTW_BILL;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve transactions where the billing date is within the last six months.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Billing date of the transaction'\n );",
    "reasoning": "The query requires filtering transactions based on the billing date. The column `COL_B` is selected because it stores the billing date for each transaction. To filter the records, the query uses a condition to check if `COL_B` is within the last six months, which can be dynamically calculated using date functions. The column `COL_A` is included to retrieve the unique transaction identifiers.",
    "action": "SELECT COL_A, COL_B AS billing_date FROM STT_RCTW_BILL WHERE COL_B >= ADD_MONTHS(CURRENT_DATE, -6);",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Aggregate the total number of transactions and the earliest billing date for each month.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Billing date of the transaction'\n );",
    "reasoning": "The query requires aggregation by month. The column `COL_B` is used to group transactions by month (using a date function to extract the month). The column `COL_A` is used to count the total number of transactions for each month. Additionally, the earliest billing date for each month is calculated using the `MIN` function on `COL_B`. These columns and aggregations satisfy the query requirements.",
    "action": "SELECT EXTRACT(MONTH FROM COL_B) AS billing_month, COUNT(COL_A) AS total_transactions, MIN(COL_B) AS earliest_billing_date FROM STT_RCTW_BILL GROUP BY EXTRACT(MONTH FROM COL_B);",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve customers who had more than 5 disputes in the last year.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_DSPT 'Tracks transaction disputes' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Customer name',\n     COL_C 'Dispute date of the transaction'\n );",
    "reasoning": "The query requires filtering customers based on the number of disputes within a specific time period (last year). The column `COL_B` is selected because it identifies the customer. The column `COL_C` is used to filter disputes to include only those within the last year. The query groups by `COL_B` to count the number of disputes per customer using `COUNT(COL_A)`. A `HAVING` clause is used to filter customers with more than 5 disputes.",
    "action": "SELECT COL_B AS customer_name, COUNT(COL_A) AS total_disputes FROM STT_RCTW_DSPT WHERE COL_C >= ADD_MONTHS(CURRENT_DATE, -12) GROUP BY COL_B HAVING COUNT(COL_A) > 5;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Calculate the average invoice amount for transactions in the last quarter.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_INVC 'Tracks invoice details' (\n     COL_A 'Unique identifier for the invoice',\n     COL_B 'Invoice amount',\n     COL_C 'Invoice date'\n );",
    "reasoning": "The query requires calculating the average invoice amount for transactions occurring in the last quarter. The column `COL_B` is selected because it stores the invoice amount, which is required for calculating the average. The column `COL_C` is used to filter transactions to include only those within the last quarter (using date functions to determine the range). These filtered results are used to calculate the average of `COL_B`.",
    "action": "SELECT AVG(COL_B) AS avg_invoice_amount FROM STT_RCTW_INVC WHERE COL_C >= TRUNC(ADD_MONTHS(CURRENT_DATE, -3), 'Q');",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve customers who made transactions worth more than $10,000 in the last month.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_TRNS 'Tracks customer transaction details' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Customer name',\n     COL_C 'Transaction amount',\n     COL_D 'Transaction date'\n );",
    "reasoning": "The query requires filtering customers based on their total transaction amount within the last month. The column `COL_B` is selected because it stores the customer name. The column `COL_C` is used to sum the transaction amounts for each customer, while the column `COL_D` is used to filter transactions to include only those within the last month. Grouping is applied on `COL_B` to calculate the total amount, and a `HAVING` clause filters customers with totals exceeding $10,000.",
    "action": "SELECT COL_B AS customer_name, SUM(COL_C) AS total_transaction_amount FROM STT_RCTW_TRNS WHERE COL_D >= ADD_MONTHS(CURRENT_DATE, -1) GROUP BY COL_B HAVING SUM(COL_C) > 10000;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all records with their payment date.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Billing date of the transaction',\n     COL_C 'Shipping date of the transaction',\n     COL_D 'Correspondence date related to the transaction',\n     COL_E 'Dispute date related to the transaction',\n     COL_F 'Invoice date issued for the transaction',\n     COL_G 'Payment date for the transaction'\n );",
    "reasoning": "The query requires all records along with their payment date. The column `COL_G` is selected because it stores the payment date of the transaction, which is the focus of this query. The column `COL_A` is included to retrieve the unique transaction identifiers. These columns together meet the query requirements.",
    "action": "SELECT COL_A, COL_G AS payment_date FROM STT_RCTW_BILL;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all records with their invoice date and dispute date.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Billing date of the transaction',\n     COL_C 'Shipping date of the transaction',\n     COL_D 'Correspondence date related to the transaction',\n     COL_E 'Dispute date related to the transaction',\n     COL_F 'Invoice date issued for the transaction',\n     COL_G 'Payment date for the transaction'\n );",
    "reasoning": "The query requires both the invoice date and dispute date for each record. The column `COL_F` is selected because it stores the invoice date, indicating when the transaction was billed. The column `COL_E` is selected because it stores the dispute date, which captures any issues raised about the transaction. Together with `COL_A` (the unique identifier), these columns fulfill the query requirements.",
    "action": "SELECT COL_A, COL_F AS invoice_date, COL_E AS dispute_date FROM STT_RCTW_BILL;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all records filtered by shipping date.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Billing date of the transaction',\n     COL_C 'Shipping date of the transaction',\n     COL_D 'Correspondence date related to the transaction',\n     COL_E 'Dispute date related to the transaction',\n     COL_F 'Invoice date issued for the transaction',\n     COL_G 'Payment date for the transaction'\n );",
    "reasoning": "The query filters transactions based on the availability of a shipping date. The column `COL_C` is selected because it stores the shipping date of the transaction. The column `COL_A` is included to provide unique identifiers for the filtered transactions. A `WHERE` clause is used to filter records where `COL_C` is not null.",
    "action": "SELECT COL_A, COL_C AS shipping_date FROM STT_RCTW_BILL WHERE COL_C IS NOT NULL;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all records sorted by correspondence date.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Billing date of the transaction',\n     COL_C 'Shipping date of the transaction',\n     COL_D 'Correspondence date related to the transaction',\n     COL_E 'Dispute date related to the transaction',\n     COL_F 'Invoice date issued for the transaction',\n     COL_G 'Payment date for the transaction'\n );",
    "reasoning": "The query requires sorting all records by correspondence date. The column `COL_D` is selected because it stores the correspondence date for the transaction, which provides insight into when communication occurred. The column `COL_A` is included as the unique identifier for the transactions. An `ORDER BY` clause is used to sort the results by `COL_D` in ascending order by default.",
    "action": "SELECT COL_A, COL_D AS correspondence_date FROM STT_RCTW_BILL ORDER BY COL_D;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve customers who made at least one payment in the last 30 days.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_BILL 'Tracks various dates and details for transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Customer name',\n     COL_G 'Payment date of the transaction'\n );",
    "reasoning": "The query requires filtering customers who made payments within the last 30 days. The column `COL_B` is selected because it identifies the customer, and the column `COL_G` is selected because it stores the payment date. A `WHERE` clause filters transactions with payment dates within the last 30 days (calculated using a date function). The `DISTINCT` keyword ensures each customer appears only once in the result.",
    "action": "SELECT DISTINCT COL_B AS customer_name FROM STT_RCTW_BILL WHERE COL_G >= CURRENT_DATE - 30;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve the total transaction amount for each customer filtered by billing date in the last year.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_TRNS 'Tracks customer transactions' (\n     COL_A 'Unique identifier for the transaction',\n     COL_B 'Customer name',\n     COL_C 'Transaction amount',\n     COL_D 'Billing date of the transaction'\n );",
    "reasoning": "The query requires the total transaction amount for each customer, but only for transactions with a billing date in the last year. The column `COL_B` is selected because it identifies the customer. The column `COL_C` is selected to sum the transaction amounts. The column `COL_D` is used to filter transactions to include only those with a billing date within the last year. Grouping by `COL_B` allows calculating the total amount for each customer.",
    "action": "SELECT COL_B AS customer_name, SUM(COL_C) AS total_transaction_amount FROM STT_RCTW_TRNS WHERE COL_D >= ADD_MONTHS(CURRENT_DATE, -12) GROUP BY COL_B;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve the average dispute resolution time for each customer.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_DSPT 'Tracks transaction disputes' (\n     COL_A 'Unique identifier for the dispute',\n     COL_B 'Customer name',\n     COL_C 'Dispute date',\n     COL_D 'Resolution date'\n );",
    "reasoning": "The query requires calculating the average time it takes to resolve disputes for each customer. The column `COL_B` is selected because it stores the customer name, while the columns `COL_C` and `COL_D` are used to calculate the resolution time by subtracting the dispute date (`COL_C`) from the resolution date (`COL_D`). Grouping by `COL_B` allows calculating the average resolution time for each customer.",
    "action": "SELECT COL_B AS customer_name, AVG(COL_D - COL_C) AS avg_resolution_time FROM STT_RCTW_DSPT WHERE COL_D IS NOT NULL GROUP BY COL_B;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve the total and average payment amounts grouped by payment date.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_PAY 'Tracks payment transactions' (\n     COL_A 'Unique identifier for the payment',\n     COL_B 'Payment amount',\n     COL_C 'Payment date'\n );",
    "reasoning": "The query requires calculating both the total and average payment amounts grouped by payment date. The column `COL_C` is selected to group transactions by payment date. The column `COL_B` is used to calculate the total and average payment amounts using the `SUM` and `AVG` functions. These aggregated results provide a summary of payments for each date.",
    "action": "SELECT COL_C AS payment_date, SUM(COL_B) AS total_payment_amount, AVG(COL_B) AS avg_payment_amount FROM STT_RCTW_PAY GROUP BY COL_C;",
    "final_answer": ""
},
{
    "sql_prompt": "You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve customers who made more than 3 payments in the last month.",
    "sql_context": "\n#### Table Definitions:\n TABLE STT_RCTW_PAY 'Tracks payment transactions' (\n     COL_A 'Unique identifier for the payment',\n     COL_B 'Customer name',\n     COL_C 'Payment date'\n );",
    "reasoning": "The query requires filtering customers based on the number of payments they made in the last month. The column `COL_B` is selected because it identifies the customer. The column `COL_C` is used to filter payments within the last month using a date condition. Grouping by `COL_B` allows counting the number of payments for each customer, and a `HAVING` clause is used to filter customers with more than 3 payments.",
    "action": "SELECT COL_B AS customer_name, COUNT(COL_A) AS payment_count FROM STT_RCTW_PAY WHERE COL_C >= ADD_MONTHS(CURRENT_DATE, -1) GROUP BY COL_B HAVING COUNT(COL_A) > 3;",
    "final_answer": ""
}

]

In [ ]:
#update the prompts JSON with new text
import json

# with open("/content/warmup_react_prompts.json", "r") as file:
#     input_json = json.load(file)

formatted_prompts = []
for entry in react_prompts:
  # if 'Retrieve' in entry['sql_prompt']:
  #       entry['sql_prompt'] = entry['sql_prompt'].replace('Retrieve', 'Find')

  # entry['sql_prompt'] = f"****Read and Understand the given question and given table or tabless and column descriptions and Write a SQL query and always use miminum 3 character aliase based on the given schema and reasoning to {entry['sql_prompt']}"

  prompt = f"""
Input: {entry['sql_prompt']}
{entry['sql_context']}

Reasoning:
{entry['reasoning']}

Action:
{entry['action']}

Final Answer:
"""
  formatted_prompts.append(prompt)


In [ ]:
formatted_prompts[0]

"\nInput: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the \n\n\n#### Question:Retrieve all records from the table containing details about employees.\n\n#### Table Definitions:\n TABLE  E_MP 'Tracks employee details' (EID 'Unique identifier for an employee', E_FN 'First Name of the employee', E_LN 'Last Name of the employee', E_S 'Indicator if the employee is active or inactive', E_J_D 'Date the employee joined');\n\nReasoning:\nThe question asks to retrieve all records from the table containing employee details. Since all details are needed, the `SELECT *` statement is appropriate to fetch all columns. The alias `emp` is assigned to the table `E_MP` for readability and ease of reference in complex queries.\n\nAction:\nSELECT * FROM E_MP emp;\n\nFinal Answer:\n"

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD token

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
import json
from IPython.display import display

df_result = []
end_token_id = tokenizer.encode(";", add_special_tokens=False)[0]
for prompt in formatted_prompts:
    #parsed_prompt = prompt.strip('{}')
    # updated_text = json.dumps(f"{parsed_json} {table_prompt}")
    print(prompt)
    tokenized_prompt = tokenize_sentences_with_t5(prompt, tokenizer)
    chunks = sliding_window_tokenizer(tokenized_prompt, 512, 256)
    decoded_chunks= []
    decoded_chunks = [tokenizer.decode(chunk, skip_special_tokens=True, clean_up_tokenization_spaces=False) for chunk in chunks]
    print(f"Decoded Chunks: {decoded_chunks}")
    print(f"chunks generated = {len(chunks)}")
    responses = []

    # Process each chunk
    for chunk in chunks:
        input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

        # Generate output
        with torch.no_grad():
            outputs = unload_model.generate(
                **input_chunk,
                max_length=256,
                num_beams=3,
                # diversity_penalty=1.2,  # Penalize repeated tokens
                # num_beam_groups=2,
    eos_token_id=tokenizer.eos_token_id,  # Stop generation at EOS token
    do_sample=True,  # Enable sampling for diversity
    # top_k=50,  # Consider top 50 tokens
    # top_p=0.9,  # Focus on tokens with 90% cumulative probability
    # temperature=0.7  # Add randomness for diverse outputs

            )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #print(f"generated output - {response}")
        responses.append(response)

    # Combine responses
    if isinstance(responses, list):
        response_text = " ".join(responses)
    print(f"generated output - {responses}")

    # Extract SQL query
    match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)

    if match:
        sql_query = match.group(0).strip()
        print(f"SQL Query: {sql_query}")

    # Execute the SQL query
    #   cursor.execute(sql_query)
    #   results = cursor.fetchall()
    #   #print("Query Results:", results)
    # except sqlite3.Error as e:
    #   print(f"Error executing SQL query: {e}")
# for i, df in enumerate(df_result, start=1):
#     # print(f"DataFrame {i}:")
#     display(df)  # Displays the DataFrame in a table format



Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the 


#### Question:Retrieve all records from the table containing details about employees.

#### Table Definitions:
 TABLE  E_MP 'Tracks employee details' (EID 'Unique identifier for an employee', E_FN 'First Name of the employee', E_LN 'Last Name of the employee', E_S 'Indicator if the employee is active or inactive', E_J_D 'Date the employee joined');

Reasoning:
The question asks to retrieve all records from the table containing employee details. Since all details are needed, the `SELECT *` statement is appropriate to fetch all columns. The alias `emp` is assigned to the table `E_MP` for readability and ease of reference in complex queries.

Action:
SELECT * FROM E_MP emp;

Final Answer:

Decoded Chunks: ["Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question:Retrieve all records from 

# **FAISS Vector Implementation**

In [ ]:
# Set up BigQuery client
#llm-text-to-sql-445914.llm_text_to_sql.RX_SALES_TRANSACTIONS
from google.cloud import bigquery
import json
client = bigquery.Client(project="llm-text-to-sql-445914")  # Replace with your project ID

def get_table_definition(project_id ="llm-text-to-sql-445914", dataset_id = "llm_text_to_sql", table_name ="RX_SALES_TRANSACTIONS"):

    query = f"""
    SELECT
        table_name,
        column_name,
        IFNULL(description, 'No Description') AS column_description
    FROM
        `{project_id}.{dataset_id}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE
        table_name = '{table_name}'
    """

    # Execute query
    query_job = client.query(query)
    df = query_job.to_dataframe()

    # Convert DataFrame to JSON format
    table_definition = {
        "table_name": table_name,
        "table_Desc": f"Details of {table_name}",
        "Columns": dict(zip(df["column_name"], df["column_description"]))
    }

    # Return JSON
    return json.dumps(table_definition, indent=4)



definition_json = get_table_definition()


In [ ]:
print(definition_json)

{
    "table_name": "RX_SALES_TRANSACTIONS",
    "table_Desc": "Details of RX_SALES_TRANSACTIONS",
    "Columns": {
        "TRX_ID": "Unique Sales transaction identifier",
        "SAL_WHS": "Warehouse Identifier",
        "SAL_WHN": "Warehouse Name",
        "SAL_WHL": "Warehouse Region",
        "SAL_WNC": "Warehouse City",
        "SAL_WHA_ONE": "Warehouse Building Address",
        "SAL_WHA_TWO": "Warehouse Street Address",
        "SAL_WHA_TRE": "Warehouse Address City",
        "SAL_WHA_FOR": "Warehouse Address State",
        "SAL_WHA_FIV": "Warehouse Address Country",
        "SAL_WHA_ZIP": "Warehouse Address zip code",
        "SAL_WHA_MGR": "Warehouse Manager Identifier",
        "SAL_WHA_MGAF": "Warehouse Manager First Name",
        "SAL_WHA_MGAL": "Warehouse Manager Last Name",
        "SAL_SALES_EID": "Sales person Employee Identifier",
        "SAL_SALES_EFN": "Sales person First Name",
        "SAL_SALES_ELN": "Sales person Last Name",
        "SAL_SALES_MID": "Sales M

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 80.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
# Example: Table structure from BigQuery
definition_dict = json.loads(definition_json)

# Initialize the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare data for embedding
table_name = definition_dict["table_name"]
columns = definition_dict["Columns"]

# Generate embeddings for each column
embeddings = []
metadata = []
for column_name, description in columns.items():
    text_representation = f"Table: {table_name}, Column: {column_name}, Description: {description}"
    embedding = model.encode(text_representation, convert_to_numpy=True)
    embeddings.append(embedding)
    metadata.append({"table_name": table_name, "column_name": column_name, "description": description})

embeddings = np.array(embeddings)

# Create FAISS index
d = embeddings.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(d)  # L2 distance for similarity search
index.add(embeddings)  # Add embeddings to the FAISS index


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#query = "Find distinct billing address"
def get_query_columns(query):
  query_embedding = model.encode(query, convert_to_numpy=True)

  # Search in FAISS
  k = 6  # Number of results to retrieve
  distances, indices = index.search(np.array([query_embedding]), k)

  # Retrieve metadata for the results
  results = [{"distance": distances[0][i], "metadata": metadata[indices[0][i]]} for i in range(k)]
  combined_texts = []
  for result in results:
      table_name = result["metadata"]["table_name"]
      columns = result["metadata"]["column_name"]
      description = result["metadata"]["description"]

      # Append table and column descriptions in the desired format
      combined_texts.append(f"{columns}: '{description}'")

  # Combine all results into a single text
  column_output = ", ".join(combined_texts)

  final_output = "#### Table Definition: \n TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (\n" + column_output + ");'\n"
  return final_output
  # print(final_output)


# distance_threshold = 1.0  # Adjust based on your use case

# # Filter results based on the threshold
# filtered_results = [
#     {"distance": distances[0][i], "metadata": metadata[indices[0][i]]}
#     for i in range(k)
#     if distances[0][i] < distance_threshold
# ]

# # Display results
# for result in results:
#     print(f"Distance: {result['distance']}")
#     print(f"Table Name: {result['metadata']['table_name']}")
#     print(f"Column Name: {result['metadata']['column_name']}")
#     print(f"Description: {result['metadata']['description']}")
#     print()

# # Display results
# for result in filtered_results:
#     print(f"Filtered Distance: {result['distance']}")
#     print(f"Filtered Table Name: {result['metadata']['table_name']}")
#     print(f"Filtered Column Name: {result['metadata']['column_name']}")
#     print(f"Filtered Description: {result['metadata']['description']}")
#     print()


In [ ]:
prompts=["Get count of sales transactions ?",
          "Get the distinct first name and last name of sales person ?",
         "Get full name of sales person",
         "Get full name of sales manager",
         "Get distinct full name of sales person",
         "Get formatted billing address of transactions",
         "get distinct formatted billing address of transactions",
         "Get distinct formatted billing address of the transactions with count of sales transactions",
         "Get sales count by billing address city",
         "Get distinct full shipping address for transactions",
          "Get the distinct wareshouse names ?",
         "Get distinct warehouse names and count of sale transactions for each warehouse",
         "Get the distinct first names for sales persons and count of sales transactions",
         "Get distinct items and their unit of measure for the transactions",
         "Get different shipping address city for the transactions",
         "Get different billing address ciry for the transactions",
         "Get different customers for the transactions for the transactions"
         "Get count of transactions group by date of sale"
         ]

In [ ]:
for prompt in prompts:
  print(prompt)
  print(get_query_columns(prompt))


Get count of sales transactions ?
#### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_ITMN: 'Sale item number used for ordering by customer', SAL_QTY: 'Sale quantity', SAL_CUST_NUM: 'Sale Customer account number', SAL_DATE: 'Date of Sale', SAL_ITEM_UOM: 'Sale item Unit of measure', SAL_INV_AMT: 'Sale invoice amount');'

Get the distinct first name and last name of sales person ?
#### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_SALES_EFN: 'Sales person First Name', SAL_SALES_ELN: 'Sales person Last Name', SAL_SALES_EID: 'Sales person Employee Identifier', SAL_SALES_MIL: 'Sales Manager Last Name', SAL_INV_ID: 'Sale invoice unique identifier', SAL_SALES_MIF: 'Sales Manager First Name');'

Get full name of sales person
#### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_SALES_ELN: 'Sales person Last Name', SAL_SALES_EFN: 'Sales person First Name', SAL_SALES

# Run test cases

In [ ]:
def replace_table_name_with_project_and_dataset(raw_query, project_id ="llm-text-to-sql-445914", dataset_id = "llm_text_to_sql"):
    """
    Replaces table_name in the query with the format 'project_id.dataset_id.table_name'.

    Parameters:
        raw_query (str): The raw SQL query string.
        project_id (str): The project ID.
        dataset_id (str): The dataset ID.

    Returns:
        str: The modified query with the table name replaced.
    """
    import re

    # Regex to identify table names after FROM, JOIN, or similar keywords
    table_pattern = re.compile(r"\bFROM\s+(\w+)\b|\bJOIN\s+(\w+)\b", re.IGNORECASE)

    def replace_table(match):
        table_name = match.group(1) or match.group(2)
        return match.group(0).replace(table_name, f"`{project_id}.{dataset_id}.{table_name}`")

    # Replace all table names with the fully qualified format
    modified_query = table_pattern.sub(replace_table, raw_query)
    return modified_query

# Example usage
raw_query = "SELECT column1, column2 FROM table_name_name_name_name_name_name WHERE column1 = 'value';"
project_id = "your_project_id"
dataset_id = "your_dataset_id"
modified_query = replace_table_name_with_project_and_dataset(raw_query, project_id, dataset_id)
print("Modified Query:")
print(modified_query)

Modified Query:
SELECT column1, column2 FROM `your_project_id.your_dataset_id.table_name_name_name_name_name_name` WHERE column1 = 'value';


In [ ]:
from google.cloud import bigquery
import pandas as pd

def execute_query_in_bigquery(query):
    """
    Executes a query in BigQuery and returns the result as a Pandas DataFrame.

    Parameters:
        query (str): The SQL query to execute.

    Returns:
        pd.DataFrame: The result of the query as a DataFrame.
    """
    # Set up BigQuery client
    client = bigquery.Client(project='llm-text-to-sql-445914')

    # Execute the query
    query_job = client.query(query)

    # Convert the result to a Pandas DataFrame
    try:
        dataframe = query_job.to_dataframe()
        print("Query executed successfully.")
        return dataframe
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
# query = "SELECT * FROM `llm-text-to-sql-445914.llm_text_to_sql.RX_SALES_TRANSACTIONS` LIMIT 10"
# result_df = execute_query_in_bigquery(query)

# if result_df is not None:
#     print("Result DataFrame:")
# #    print(result_df)
# result_df

In [ ]:
# Set up BigQuery client
#llm-text-to-sql-445914.llm_text_to_sql.RX_SALES_TRANSACTIONS
client = bigquery.Client(project="llm-text-to-sql-445914")  # Replace with your project ID

def get_table_definition(project_id ="llm-text-to-sql-445914", dataset_id = "llm_text_to_sql", table_name ="RX_SALES_TRANSACTIONS"):
    """
    Retrieves table definition including table name, column names, and column descriptions.

    Parameters:
        project_id (str): Google Cloud project ID.
        dataset_id (str): Dataset ID in BigQuery.
        table_name (str): Table name in BigQuery.

    Returns:
        pd.DataFrame: DataFrame containing column name and description.
    """
    query = f"""
    SELECT
        table_name,
        column_name,
        IFNULL(description, 'No Description') AS column_description
    FROM
        `{project_id}.{dataset_id}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE
        table_name = '{table_name}'
    """

    # Execute query
    query_job = client.query(query)
    df = query_job.to_dataframe()

    # Format as `column_name : 'Description'`
    df["formatted"] = df.apply(
        lambda row: f"{row['column_name']} : '{row['column_description']}'", axis=1
    )

    return df[["table_name", "formatted"]]

def combine_formatted_strings(df):
    """
    Combines the 'formatted' column of a DataFrame into a single string separated by commas.

    Parameters:
        df (pd.DataFrame): DataFrame containing a 'formatted' column.

    Returns:
        str: Combined string of all formatted entries separated by commas.
    """
    return ", ".join(df["formatted"].tolist())

definition_df = get_table_definition()
combined_string = ""
if not definition_df.empty:
    combined_string = combine_formatted_strings(definition_df)
    print(f"Combined String: {combined_string}")
    #print(combined_string)
else:
    print("No table definition found.")
table_prompt = "Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the table definition\n #### Table Definition: \n TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (\n" + combined_string + ");'\n" + "###Constraint: Use only the information provided in the Table Definition"

Combined String: TRX_ID : 'Unique Sales transaction identifier', SAL_WHS : 'Warehouse Identifier', SAL_WHN : 'Warehouse Name', SAL_WHL : 'Warehouse Region', SAL_WNC : 'Warehouse City', SAL_WHA_ONE : 'Warehouse Building Address', SAL_WHA_TWO : 'Warehouse Street Address', SAL_WHA_TRE : 'Warehouse Address City', SAL_WHA_FOR : 'Warehouse Address State', SAL_WHA_FIV : 'Warehouse Address Country', SAL_WHA_ZIP : 'Warehouse Address zip code', SAL_WHA_MGR : 'Warehouse Manager Identifier', SAL_WHA_MGAF : 'Warehouse Manager First Name', SAL_WHA_MGAL : 'Warehouse Manager Last Name', SAL_SALES_EID : 'Sales person Employee Identifier', SAL_SALES_EFN : 'Sales person First Name', SAL_SALES_ELN : 'Sales person Last Name', SAL_SALES_MID : 'Sales Manager Identifier', SAL_SALES_MIF : 'Sales Manager First Name', SAL_SALES_MIL : 'Sales Manager Last Name', SAL_SALES_CP : 'Sales person commission percentage', SAL_SALES_HD : 'Sales person hire date', SAL_SSIC : 'Sale item unique identifier', SAL_ITMN : 'Sale i

In [ ]:
def format_datediff_to_bigquery(query):
    """
    Convert SQL queries with DATEDIFF(end, start) into BigQuery's DATE_DIFF(end, start, 'DAY').
    """
    # Regex to match DATEDIFF(end, start)
    formatted_query = query
    pattern = r"DATEDIFF\((\w+),\s*(\w+)\)"

    # Replace with BigQuery's DATE_DIFF
    formatted_query = re.sub(pattern, r"DATE_DIFF(\1, \2, DAY)", query)

    return formatted_query

In [ ]:
def format_to_bigquery(query):
    """
    Format a SQL query for BigQuery compatibility.
    Ensures proper use of backticks, dataset.table notation, and BigQuery-specific functions.
    """
    # Replace table references with BigQuery's dataset.table format
    query = query.replace("[", "`").replace("]", "`")  # Replace square brackets with backticks

    # Replace non-standard functions with BigQuery equivalents
    query = query.replace("ISNULL", "IFNULL")  # Replace ISNULL with IFNULL
    query = query.replace("GETDATE()", "CURRENT_TIMESTAMP()")  # Replace GETDATE with BigQuery's CURRENT_TIMESTAMP
    query = query.replace("LEN(", "LENGTH(")  # Replace LEN with LENGTH for strings
    query = query.replace("COUNT(*)", "COUNT(1)")  # Replace COUNT(*) with COUNT(1)
    query = format_datediff_to_bigquery(query)

    # Add other function replacements as needed
    # BigQuery-specific adjustments for table references or queries can be done here

    return query

In [ ]:
pre_tokenized_table = tokenize_sentences_with_t5(table_prompt, tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (1264 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
len(pre_tokenized_table)

1274

In [ ]:
prompts=[
    #"Get count of sales transactions ?",
    #      "Get the distinct first name and last name of sales person ?",
         "Get full name of sales persons",
         "Get full name of sales manager",
         "Get distinct full name of sales person",
         "Get formatted billing address of transactions",
         "get distinct formatted billing address of transactions",
         "Get distinct formatted billing address of the transactions with count of sales transactions",
         "Get sales count by billing address city",
         "Get distinct full shipping address for transactions",
          "Get the distinct wareshouse names ?",
         "Get distinct warehouse names and count of sale transactions for each warehouse",
         "Get the distinct first names for sales persons and count of sales transactions",
         "Get distinct items number and their unit of measure for the transactions",
         "Get different shipping address city for the transactions",
         "Get different billing address ciry for the transactions",
         "Get different customers for the transactions for the transactions"
         "Get count of transactions group by date of sale"
         ]

In [ ]:
import json
from IPython.display import display

df_result = []
for prompt in prompts:
    #parsed_prompt = prompt.strip('{}')
    # updated_text = json.dumps(f"{parsed_json} {table_prompt}")
    vector_prompt = get_query_columns(prompt)
    prepend_text = "Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: "
    full_prompt = f"{prepend_text} {prompt} {vector_prompt}"
    print(full_prompt)
    tokenized_prompt = tokenize_sentences_with_t5(full_prompt, tokenizer)
    #end_token_id = tokenizer.encode(";", add_special_tokens=False)[0]
    #tokenized_test = tokenize_sentences_with_t5("This is One", tokenizer)
    #combined_tokenized_input = combine_t5_tokenized_strings(pre_tokenized_table, tokenized_prompt, tokenizer)
    #combined_tokenized_input = combine_t5_tokenized_strings(tokenized_prompt,pre_tokenized_table, tokenizer)

    #combined_prompt = combine_t5_tokenized_strings(pre_tokenized_table, tokenized_prompt, tokenizer)
    #chunks = sliding_window_tokenizer(combined_tokenized_input, 512, 256)
    chunks = sliding_window_tokenizer(tokenized_prompt, 512, 256)
    print(f"chunks generated = {len(chunks)}")
    decoded_chunks= []
    decoded_chunks = [tokenizer.decode(chunk, skip_special_tokens=True, clean_up_tokenization_spaces=False) for chunk in chunks]
    print(f"Decoded Chunks: {decoded_chunks}")
    print(f"chunks generated = {len(chunks)}")
    responses = []

    # Process each chunk
    for chunk in chunks:
        input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

        # Generate output
        with torch.no_grad():
            outputs = unload_model.generate(
                **input_chunk,
                max_length=1024,
                num_beams=5,
                # diversity_penalty=1.2,  # Penalize repeated tokens
                # num_beam_groups=2,
                #eos_token_id=end_token_id,
    eos_token_id=tokenizer.eos_token_id,  # Stop generation at EOS token
    do_sample=True,  # Enable sampling for diversity
    # top_k=50,  # Consider top 50 tokens
    # top_p=0.9,  # Focus on tokens with 90% cumulative probability
    # temperature=0.7  # Add randomness for diverse outputs

            )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #print(f"generated output - {response}")
        responses.append(response)

    # Combine responses
    if isinstance(responses, list):
        response_text = " ".join(responses)
    #print(f"generated output - {responses}")

    # Extract SQL query
    match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)

    if match:
        sql_query = match.group(0).strip()
        #display_results_as_dataframe(cursor,sql_query)
        modified_query = replace_table_name_with_project_and_dataset(sql_query)
        print(f"Modified Query: {modified_query}")
        r_df = execute_query_in_bigquery(modified_query)

        if r_df is not None:
          df_result.append(r_df)
          display(r_df)

    # Execute the SQL query
    #   cursor.execute(sql_query)
    #   results = cursor.fetchall()
    #   #print("Query Results:", results)
    # except sqlite3.Error as e:
    #   print(f"Error executing SQL query: {e}")
# for i, df in enumerate(df_result, start=1):
#     # print(f"DataFrame {i}:")
#     display(df)  # Displays the DataFrame in a table format


Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question:  Get full name of sales persons #### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_SALES_ELN: 'Sales person Last Name', SAL_SALES_EFN: 'Sales person First Name', SAL_SALES_MIL: 'Sales Manager Last Name', SAL_CUST_ID: 'Sale Customer Identifier', SAL_SALES_MIF: 'Sales Manager First Name', SAL_SALES_EID: 'Sales person Employee Identifier');'

chunks generated = 1
Decoded Chunks: ["Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: Get full name of sales persons #### Table Definition: TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details ( SAL_SALES_ELN: 'Sales person Last Name', SAL_SALES_EFN: 'Sales person First Name', SAL_SALES_MIL: 'Sales Manager Last Name', SAL_CUST_ID: 'Sale Customer Identifier', SAL_SALES_MIF: 

,full_name
0,Employee_Last_93
1,Employee_Last_7
2,Employee_Last_93
3,Employee_Last_31
4,Employee_Last_11
5,Employee_Last_64
6,Employee_Last_62
7,Employee_Last_16
8,Employee_Last_75
9,Employee_Last_46


Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question:  Get full name of sales manager #### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_SALES_MIL: 'Sales Manager Last Name', SAL_SALES_MIF: 'Sales Manager First Name', SAL_WHA_MGAL: 'Warehouse Manager Last Name', SAL_SALES_ELN: 'Sales person Last Name', SAL_WHA_MGAF: 'Warehouse Manager First Name', SAL_SALES_EFN: 'Sales person First Name');'

chunks generated = 1
Decoded Chunks: ["Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: Get full name of sales manager #### Table Definition: TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details ( SAL_SALES_MIL: 'Sales Manager Last Name', SAL_SALES_MIF: 'Sales Manager First Name', SAL_WHA_MGAL: 'Warehouse Manager Last Name', SAL_SALES_ELN: 'Sales person Last Name', SAL_WHA_MGA

,sales_manager_full_name
0,Manager_Last_9
1,Manager_Last_26
2,Manager_Last_36
3,Manager_Last_45
4,Manager_Last_45
5,Manager_Last_21
6,Manager_Last_14
7,Manager_Last_26
8,Manager_Last_32
9,Manager_Last_12


Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question:  Get distinct full name of sales person #### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_SALES_ELN: 'Sales person Last Name', SAL_SALES_EFN: 'Sales person First Name', SAL_SALES_EID: 'Sales person Employee Identifier', SAL_INV_ID: 'Sale invoice unique identifier', SAL_SALES_MIL: 'Sales Manager Last Name', SAL_CUST_ID: 'Sale Customer Identifier');'

chunks generated = 1
Decoded Chunks: ["Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: Get distinct full name of sales person #### Table Definition: TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details ( SAL_SALES_ELN: 'Sales person Last Name', SAL_SALES_EFN: 'Sales person First Name', SAL_SALES_EID: 'Sales person Employee Identifier', SAL_INV_ID: 'Sale invoice un

...

,warehouse_address_city
0,City_C
1,City_B
2,City_C
3,City_C
4,City_A
5,City_C
6,City_B
7,City_A
8,City_B
9,City_B


Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question:  Get different billing address ciry for the transactions #### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_CUST_A5: 'Sale Customer Billing address Country', SAL_CUST_A4: 'Sale Customer Billing address state', SAL_CUST_A6: 'Sale Customer Billing address zip code', SAL_CUST_A3: 'Sale Customer Billing address city', SAL_CUST_A2: 'Sale Customer Billing address Street', SAL_CUST_A1: 'Sale Customer Billing address House number');'

chunks generated = 1
Decoded Chunks: ["Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: Get different billing address ciry for the transactions #### Table Definition: TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details ( SAL_CUST_A5: 'Sale Customer Billing address Country', SAL_CUST_A4: 

,billing_address,billing_address_1,billing_house_number
0,City_C,Street_42,House_55
1,City_B,Street_36,House_46
2,City_C,Street_16,House_76
3,City_C,Street_1,House_15
4,City_C,Street_8,House_37
5,City_B,Street_30,House_21
6,City_C,Street_28,House_26
7,City_B,Street_5,House_73
8,City_B,Street_9,House_34
9,City_A,Street_29,House_10


Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question:  Get different customers for the transactions for the transactionsGet count of transactions group by date of sale #### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_CUST_NUM: 'Sale Customer account number', SAL_DATE: 'Date of Sale', SAL_CUST_ACCT: 'Sale Customer Account Identifier', SAL_DEL_DATE: 'Delivery date of sale', SAL_ITMN: 'Sale item number used for ordering by customer', SAL_CUST_ID: 'Sale Customer Identifier');'

chunks generated = 1
Decoded Chunks: ["Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: Get different customers for the transactions for the transactionsGet count of transactions group by date of sale #### Table Definition: TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details ( SAL_CUST_NUM: 

,SAL_CUST_ACCT,SAL_DEL_DATE,TRANSACTION_COUNT
0,Account_75,2025-01-17,1
1,Account_75,2025-01-18,1
2,Account_66,2025-01-15,1
3,Account_88,2025-01-13,1
4,Account_60,2025-01-14,1
5,Account_28,2025-01-14,1
6,Account_93,2025-01-21,1
7,Account_61,2025-01-19,1
8,Account_31,2025-01-15,1
9,Account_40,2025-01-20,1


In [ ]:
failed_prompts = { "Get full billing city of all open invoices. ###CONSTRAINT: Use column name provided in Table Definition only"
}

import json
from IPython.display import display

df_result = []
for prompt in failed_prompts:
    #parsed_prompt = prompt.strip('{}')
    # updated_text = json.dumps(f"{parsed_json} {table_prompt}")
    vector_prompt = get_query_columns(prompt)
    prepend_text = "Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: "
    full_prompt = f"{prepend_text} {prompt} {vector_prompt}"
    print(full_prompt)
    tokenized_prompt = tokenize_sentences_with_t5(full_prompt, tokenizer)
    #end_token_id = tokenizer.encode(";", add_special_tokens=False)[0]
    #tokenized_test = tokenize_sentences_with_t5("This is One", tokenizer)
    #combined_tokenized_input = combine_t5_tokenized_strings(pre_tokenized_table, tokenized_prompt, tokenizer)
    #combined_tokenized_input = combine_t5_tokenized_strings(tokenized_prompt,pre_tokenized_table, tokenizer)

    #combined_prompt = combine_t5_tokenized_strings(pre_tokenized_table, tokenized_prompt, tokenizer)
    #chunks = sliding_window_tokenizer(combined_tokenized_input, 512, 256)
    chunks = sliding_window_tokenizer(tokenized_prompt, 512, 256)
    print(f"chunks generated = {len(chunks)}")
    decoded_chunks= []
    decoded_chunks = [tokenizer.decode(chunk, skip_special_tokens=True, clean_up_tokenization_spaces=False) for chunk in chunks]
    print(f"Decoded Chunks: {decoded_chunks}")
    print(f"chunks generated = {len(chunks)}")
    responses = []

    # Process each chunk
    for chunk in chunks:
        input_chunk = {'input_ids': torch.tensor(chunk).unsqueeze(0).cuda()}

        # Generate output
        with torch.no_grad():
            outputs = unload_model.generate(
                **input_chunk,
                max_length=1024,
                num_beams=5,
                # diversity_penalty=1.2,  # Penalize repeated tokens
                # num_beam_groups=2,
                #eos_token_id=end_token_id,
    eos_token_id=tokenizer.eos_token_id,  # Stop generation at EOS token
    do_sample=True,  # Enable sampling for diversity
    # top_k=50,  # Consider top 50 tokens
    # top_p=0.9,  # Focus on tokens with 90% cumulative probability
    # temperature=0.7  # Add randomness for diverse outputs

            )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #print(f"generated output - {response}")
        responses.append(response)

    # Combine responses
    if isinstance(responses, list):
        response_text = " ".join(responses)
    print(f"generated output - {responses}")

    # Extract SQL query
    match = re.search(r"SELECT.*?;", response_text, re.IGNORECASE | re.DOTALL)

    if match:
        sql_query = match.group(0).strip()
        #display_results_as_dataframe(cursor,sql_query)
        modified_query = format_to_bigquery(replace_table_name_with_project_and_dataset(sql_query))
        print(f"Modified Query: {modified_query}")
        r_df = execute_query_in_bigquery(modified_query)

        if r_df is not None:
          df_result.append(r_df)
          display(r_df)

    # Execute the SQL query
    #   cursor.execute(sql_query)
    #   results = cursor.fetchall()
    #   #print("Query Results:", results)
    # except sqlite3.Error as e:
    #   print(f"Error executing SQL query: {e}")
# for i, df in enumerate(df_result, start=1):
#     # print(f"DataFrame {i}:")
#     display(df)  # Displays the DataFrame in a table format


Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question:  Get full billing city of all open invoices. ###CONSTRAINT: Use column name provided in Table Definition only #### Table Definition: 
 TABLE RX_SALES_TRANSACTIONS :'Contains sales transaction details (
SAL_CUST_A3: 'Sale Customer Billing address city', SAL_INV_AMT: 'Sale invoice amount', SAL_CUST_A5: 'Sale Customer Billing address Country', SAL_CUST_A1: 'Sale Customer Billing address House number', SAL_CUST_A6: 'Sale Customer Billing address zip code', SAL_INVOICE_STS: 'Sale invoice status ('O', 'C')');'

chunks generated = 1
Decoded Chunks: ["Input: You are a SQL assistant. Generate SQL queries based on the user's request, using only the columns provided in the #### Question: Get full billing city of all open invoices. ###CONSTRAINT: Use column name provided in Table Definition only #### Table Definition: TABLE RX_SALES_TRANSACTIONS :'Contains sales t

,full_billing_city
0,City_B
1,City_C
2,City_B
3,City_C
4,City_A
5,City_A
6,City_C
7,City_C
8,City_C


In [ ]:
display(df_result[0])

,TRX_ID,SAL_WHS,SAL_WHN,SAL_WHL,SAL_WNC,SAL_WHA_ONE,SAL_WHA_TWO,SAL_WHA_TRE,SAL_WHA_FOR,SAL_WHA_FIV,...,SAL_CUST_A6,SAL_CUST_B1,SAL_CUST_B2,SAL_CUST_B3,SAL_CUST_B4,SAL_CUST_B5,SAL_CUST_B6,SAL_QTY,SAL_DATE,SAL_DEL_DATE
0,1,5,Warehouse_3,South,City_B,Building_75,Street_24,City_C,State_X,Country_ABC,...,78901,House_67,Street_24,City_A,State_X,Country_ABC,23958,45,2022-08-02,2025-01-17
1,4,9,Warehouse_4,East,City_B,Building_11,Street_23,City_C,State_Y,Country_ABC,...,35240,House_56,Street_37,City_B,State_Y,Country_ABC,24746,17,2022-12-16,2025-01-13
2,5,5,Warehouse_3,East,City_A,Building_37,Street_36,City_C,State_X,Country_ABC,...,82038,House_49,Street_10,City_A,State_X,Country_ABC,46616,83,2023-09-22,2025-01-14
3,9,1,Warehouse_3,West,City_B,Building_73,Street_8,City_A,State_X,Country_ABC,...,68410,House_67,Street_22,City_A,State_Y,Country_ABC,51979,93,2023-04-18,2025-01-15
4,10,5,Warehouse_7,East,City_B,Building_50,Street_47,City_A,State_Z,Country_ABC,...,57197,House_20,Street_10,City_C,State_Z,Country_ABC,29926,98,2023-12-09,2025-01-20
5,11,3,Warehouse_3,East,City_A,Building_76,Street_27,City_B,State_X,Country_ABC,...,51460,House_62,Street_37,City_A,State_X,Country_ABC,58316,96,2024-10-11,2025-01-15
6,13,5,Warehouse_5,East,City_A,Building_98,Street_45,City_C,State_Y,Country_ABC,...,20724,House_61,Street_19,City_C,State_Z,Country_ABC,15990,95,2022-09-11,2025-01-19
7,14,7,Warehouse_4,South,City_A,Building_36,Street_43,City_B,State_X,Country_ABC,...,92889,House_33,Street_49,City_C,State_Y,Country_ABC,85866,96,2023-01-26,2025-01-17
8,17,8,Warehouse_2,West,City_A,Building_67,Street_2,City_A,State_Z,Country_ABC,...,16541,House_35,Street_21,City_C,State_Z,Country_ABC,97314,16,2022-10-03,2025-01-15
9,19,1,Warehouse_6,West,City_C,Building_2,Street_28,City_B,State_Y,Country_ABC,...,91137,House_92,Street_20,City_A,State_X,Country_ABC,13523,8,2023-08-21,2025-01-20
